In [1]:
import pandas as pd
import numpy as np
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

from datetime import date as dt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import NearMiss
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [2]:
com = pd.read_csv('companies.csv')

In [3]:
data = com.drop(columns = ['id','entity_type','permalink','region','created_by','Unnamed: 0.1','normalized_name','parent_id','domain','homepage_url','logo_url','logo_width','logo_height','overview','short_description','twitter_username','short_description','description','tag_list','state_code','city','first_investment_at', 'last_investment_at','first_milestone_at','last_milestone_at',  'first_funding_at','last_funding_at','lat','lng'])
data.dropna(subset= ['name','country_code','category_code','founded_at'],inplace = True)
data['investment_rounds'] = data['investment_rounds'].fillna(data['investment_rounds'].median())
data['invested_companies'] = data['invested_companies'].fillna(data['invested_companies'].median())
data['funding_rounds'] = data['funding_rounds'].fillna(data['funding_rounds'].median())
data['funding_total_usd'] = data['funding_total_usd'].fillna(data['funding_total_usd'].mean())
data['milestones'] = data['milestones'].fillna(data['milestones'].mode()[0])
data['relationships'] = data['relationships'].fillna(data['relationships'].mean())
data['ROI'] = data['ROI'].fillna(data['ROI'].mean())

In [4]:
data['founded_at'] = pd.to_datetime(data['founded_at'])
data['founded_at'] = data['founded_at'].dt.year

data['founded_at'] = data['founded_at'].fillna(data['founded_at'].mode()[0])
data['closed_at'] = pd.to_datetime(data['closed_at'])
data['closed_at'] = data['closed_at'].dt.year

In [5]:
for i in data['status']:
    if (i == 'operating' or 'ipo'):
        data['closed_at'].fillna(2021,inplace = True)
    elif (i == 'accquired' or 'closed'):
        data['closed_at'].fillna(2012, inplace = True)

In [6]:
created = []
for i in data['created_at']:
    created.append(i.split('-')[0])
data['created_at']= created

data['created_at'] = pd.to_datetime(data['created_at'])
data['created_at'] = data['created_at'].dt.year

In [7]:
created = []
for i in data['updated_at']:
    created.append(int(i.split('-')[0]))
data['updated_at']= created

In [8]:
data['companies_age'] = data['closed_at'] - data['founded_at']

index = np.where(data['companies_age'] < 0)
data['companies_age'] = data['companies_age'].drop(data['companies_age'].index[index])

data['companies_age'] = data['companies_age'].fillna(0)

In [9]:
data.reset_index(inplace = True)

In [10]:
q1 = data.quantile(0.25)
q3 = data.quantile(0.75)
iqr = q3 - q1

high = (q3.funding_total_usd + (1.5 * iqr.funding_total_usd))
index = np.where(data['funding_total_usd'] > high)
data = data.drop(data.index[index])
data.reset_index()

high_fu = (q3.funding_rounds + (1.5 * iqr.funding_rounds))
index_fu = np.where(data['funding_rounds'] > high)
data = data.drop(data.index[index_fu])
data.reset_index()

,level_0,index,entity_id,name,category_code,status,founded_at,closed_at,country_code,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,created_at,updated_at,ROI,companies_age
0,1,5,10002,Synergie Media,advertising,operating,2007,2021.0,MAR,1.0,1.0,1.0,1.581978e+07,1.0,2.00000,2008,2008,14.41045,14.0
1,2,6,10003,Green Basti Ecosystems,cleantech,operating,2008,2021.0,IND,1.0,1.0,1.0,1.581978e+07,1.0,4.43465,2008,2010,14.41045,13.0
2,3,12,10009,dimension5 labs,advertising,operating,2008,2021.0,USA,1.0,1.0,1.0,1.581978e+07,1.0,2.00000,2008,2008,14.41045,13.0
3,4,13,1001,FriendFeed,web,acquired,2007,2021.0,USA,1.0,1.0,1.0,5.000000e+06,3.0,14.00000,2007,2013,9.50000,14.0
4,5,15,10011,PoetryVisualized.com,games_video,operating,2008,2021.0,USA,1.0,1.0,1.0,1.581978e+07,1.0,3.00000,2008,2009,14.41045,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60064,64089,196548,99940,Webgility,ecommerce,operating,2007,2021.0,USA,1.0,1.0,1.0,1.581978e+07,2.0,5.00000,2011,2013,14.41045,14.0
60065,64090,196549,9995,FohBoh,public_relations,operating,2007,2021.0,USA,1.0,1.0,1.0,7.500000e+05,1.0,14.00000,2008,2013,14.41045,14.0
60066,64091,196550,9996,CSC,consulting,operating,1959,2021.0,USA,1.0,1.0,1.0,1.581978e+07,3.0,44.00000,2008,2013,14.41045,62.0
60067,64092,196551,9997,Top-candidate,search,operating,2008,2021.0,USA,1.0,1.0,1.0,1.581978e+07,1.0,1.00000,2008,2013,14.41045,13.0


In [11]:
top = ['USA','GBR','IND','CAN','DEU','FRA','AUS','ESP','ISR','NLD']
data.loc[~data['country_code'].isin(top),'country_code'] = 'other'

data['category_code'] = data['category_code'].replace(['public_relations','hardware','network_hosting','education','search','cleantech','analytics','finance','social','security','health','medical','semiconductor','legal','travel','news','manufacturing','fashion','hospitality','real_estate','photo_video','messaging','music','local','sports','transportation','nonprofit','automotive','design','nanotech','pets','government'],'other')
data = pd.get_dummies(data, columns = ['category_code','country_code'])

In [12]:
data.drop(columns = ['name','invested_companies'],inplace = True)

In [13]:
data['status'] = data['status'].replace('operating',1)
data['status'] = data['status'].replace('ipo',1)
data['status'] = data['status'].replace('closed',0)
data['status'] = data['status'].replace('acquired',0)

In [14]:
x = data.drop('status',axis = 1)
y = data.status

In [15]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.25, random_state = 4)
ds = NearMiss(0.75)
x_train_nm,y_train_nm = ds.fit_resample(x_train,y_train)
y_train_nm = y

In [16]:
data = data.drop(columns = ['index','entity_id'])

In [17]:
x = data.drop('status',axis = 1)
y = y_train_nm

In [18]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [19]:
'''randomForest = RandomForestClassifier(n_estimators = 100, min_samples_split = 10, min_samples_leaf = 2, max_features= 'auto',max_depth = 18)
x_train,x_test,y_train,y_test = train_test_split(x_scaled, y, test_size = 0.25, random_state = 8)
randomForest.fit(x_train,y_train)'''

x_train,x_test,y_train,y_test = train_test_split(x_scaled, y, test_size = 0.25, random_state = 8)

pipeline_randomforest =  Pipeline([('scaler1', StandardScaler()),
                               ('rf_classifier',RandomForestClassifier(n_estimators = 100, min_samples_split = 10, min_samples_leaf = 2, max_features= 'auto',max_depth = 18))])
pipeline_randomforest.fit(x_train,y_train)

Pipeline(steps=[('scaler1', StandardScaler()),
                ('rf_classifier',
                 RandomForestClassifier(max_depth=18, max_features='auto',
                                        min_samples_leaf=2,
                                        min_samples_split=10))])

In [20]:
from joblib import dump
dump(pipeline_randomforest,'./../savedModel/model.joblib')

['./../savedModel/model.joblib']